## Naive Bayes Theorem

## Naive Bayes Classification Example

Let's consider a dataset with the following features:
- **Outlook** (Sunny, Overcast, Rain)
- **Humidity** (High, Normal)
- **Windy** (True, False)
- **Play** (Yes, No)

Suppose our dataset is:

| Outlook  | Humidity | Windy | Play |
|----------|----------|-------|------|
| Sunny    | High     | False | No   |
| Sunny    | High     | True  | No   |
| Overcast | High     | False | Yes  |
| Rain     | Normal   | False | Yes  |
| Rain     | Normal   | True  | No   |
| Rain     | High     | False | Yes  |
| Overcast | Normal   | False | Yes  |
| Sunny    | Normal   | False | Yes  |
| Sunny    | Normal   | True  | Yes  |
| Rain     | High     | True  | No   |
| Sunny    | High     | False | No   |
| Overcast | High     | True  | Yes  |
| Overcast | Normal   | True  | Yes  |
| Rain     | Normal   | False | Yes  |

### Prediction for: Outlook='Sunny', Humidity='High', Windy='False'

---

### **Step 1: Calculate Prior Probability**

- **P(Yes)** = (Number of Yes) / (Total)
- **P(No)** = (Number of No) / (Total)

Suppose:
- Number of Yes = 9
- Number of No = 5
- Total = 14

So,
- **P(Yes) = 9/14**
- **P(No) = 5/14**

---

### **Step 2: Calculate Likelihoods**

For each class (Yes/No), calculate the likelihood of the features:

#### For **Yes**:
- P(Outlook=Sunny | Yes) = (Sunny & Yes) / (Yes) = 2/9
- P(Humidity=High | Yes) = (High & Yes) / (Yes) = 3/9
- P(Windy=False | Yes) = (False & Yes) / (Yes) = 6/9

#### For **No**:
- P(Outlook=Sunny | No) = (Sunny & No) / (No) = 3/5
- P(Humidity=High | No) = (High & No) / (No) = 4/5
- P(Windy=False | No) = (False & No) / (No) = 2/5

---

### **Step 3: Final Prediction**

Calculate the unnormalized probabilities:

- **P(Yes | X) ∝ P(Yes) × P(Outlook=Sunny|Yes) × P(Humidity=High|Yes) × P(Windy=False|Yes)**
- **P(No | X) ∝ P(No) × P(Outlook=Sunny|No) × P(Humidity=High|No) × P(Windy=False|No)**

Plug in the values:

- P(Yes | X) ∝ (9/14) × (2/9) × (3/9) × (6/9)
- P(No | X) ∝ (5/14) × (3/5) × (4/5) × (2/5)

Calculate:

- P(Yes | X) ∝ (9/14) × (2/9) × (3/9) × (6/9) = (9×2×3×6) / (14×9×9×9) = 324 / 10206 ≈ 0.0318
- P(No | X) ∝ (5/14) × (3/5) × (4/5) × (2/5) = (5×3×4×2) / (14×5×5×5) = 120 / 1750 ≈ 0.0686

---

### **Step 4: Normalize the Probabilities**

Sum = 0.0318 + 0.0686 = 0.1004

- **P(Yes | X) = 0.0318 / 0.1004 ≈ 0.317**
- **P(No | X) = 0.0686 / 0.1004 ≈ 0.683**

---

### **Conclusion**

Since **P(No | X) > P(Yes | X)**, the prediction is **No** (the user will not play).

In [1]:
import pandas as pd 
from collections import defaultdict

class NaiveBayes:
    def __init__(self):
        self.prior_probs = {}
        self.likelihoods = defaultdict(dict)
        self.classes = []

    def fit(self, X, y):
        """
        Fit the Naive Bayes model to the dataset.
        :param X: DataFrame of features
        :param y: Series of target labels
        """
        self.classes = y.unique()
        total_samples = len(y)

        print("Classes:", self.classes)
        print("Total samples:", total_samples)

        # Calculate prior probabilities
        for cls in self.classes:
            self.prior_probs[cls] = len(y[y == cls]) / total_samples
            print(f"Prior probability P({cls}): {self.prior_probs[cls]}")

        # Calculate likelihoods
        for feature in X.columns:
            print(f"\nProcessing feature: {feature}")
            for cls in self.classes:
                feature_values = X[feature][y == cls]
                total_cls_samples = len(feature_values)
                value_counts = feature_values.value_counts()

                print(f"  Class: {cls}")
                print(f"  Total samples for class {cls}: {total_cls_samples}")
                print(f"  Value counts for feature '{feature}' in class {cls}:")
                print(value_counts)

                for value, count in value_counts.items():
                    self.likelihoods[cls][(feature, value)] = count / total_cls_samples
                    print(f"    Likelihood P({feature}={value} | {cls}): {self.likelihoods[cls][(feature, value)]}")

        
    def predict(self, X):
        """
        Predict the class for each sample in X.
        :param X: DataFrame of features
        :return: List of predicted classes
        """
        predictions = []
        for _, sample in X.iterrows():
            class_probs = {}
            for cls in self.classes:
                # Start with the prior probability
                class_probs[cls] = self.prior_probs[cls]

                # Multiply by the likelihoods
                for feature, value in sample.items():
                    class_probs[cls] *= self.likelihoods[cls].get((feature, value), 1e-6)  # Smoothing for unseen values

            # Normalize probabilities
            total_prob = sum(class_probs.values())
            for cls in class_probs:
                class_probs[cls] /= total_prob

            # Choose the class with the highest probability
            predictions.append(max(class_probs, key=class_probs.get))

        return predictions



In [2]:
# Dataset
data = {
    "Outlook": ["Sunny", "Sunny", "Overcast", "Rain", "Rain", "Rain", "Overcast", "Sunny", "Sunny", "Rain", "Sunny", "Overcast", "Overcast", "Rain"],
    "Humidity": ["High", "High", "High", "Normal", "Normal", "High", "Normal", "Normal", "Normal", "High", "High", "High", "Normal", "Normal"],
    "Windy": [False, True, False, False, True, False, False, False, True, True, False, True, True, False],
    "Play": ["No", "No", "Yes", "Yes", "No", "Yes", "Yes", "Yes", "Yes", "No", "No", "Yes", "Yes", "Yes"]
}

df = pd.DataFrame(data)
X = df[["Outlook", "Humidity", "Windy"]]
y = df["Play"]

# Train the model
model = NaiveBayes()
model.fit(X, y)

Classes: ['No' 'Yes']
Total samples: 14
Prior probability P(No): 0.35714285714285715
Prior probability P(Yes): 0.6428571428571429

Processing feature: Outlook
  Class: No
  Total samples for class No: 5
  Value counts for feature 'Outlook' in class No:
Outlook
Sunny    3
Rain     2
Name: count, dtype: int64
    Likelihood P(Outlook=Sunny | No): 0.6
    Likelihood P(Outlook=Rain | No): 0.4
  Class: Yes
  Total samples for class Yes: 9
  Value counts for feature 'Outlook' in class Yes:
Outlook
Overcast    4
Rain        3
Sunny       2
Name: count, dtype: int64
    Likelihood P(Outlook=Overcast | Yes): 0.4444444444444444
    Likelihood P(Outlook=Rain | Yes): 0.3333333333333333
    Likelihood P(Outlook=Sunny | Yes): 0.2222222222222222

Processing feature: Humidity
  Class: No
  Total samples for class No: 5
  Value counts for feature 'Humidity' in class No:
Humidity
High      4
Normal    1
Name: count, dtype: int64
    Likelihood P(Humidity=High | No): 0.8
    Likelihood P(Humidity=Normal 

In [3]:
# Predict for a new sample
sample = pd.DataFrame([{"Outlook": "Sunny", "Humidity": "High", "Windy": False}])
prediction = model.predict(sample)
print("Prediction:", prediction)

Prediction: ['No']
